In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='8f23d16a-bcec-44fd-b57e-089fffb149d7', project_access_token='p-98606b388205865c63e4803f5e31406948635405')
pc = project.project_context


# MARATONA BEHIND THE CODE 2020

## DESAFIO 6 - LIT

<hr>

## Installing Libs

In [ ]:
!pip install scikit-learn --upgrade

In [ ]:
!pip install xgboost --upgrade

In [ ]:
!pip install imblearn --upgrade

In [ ]:
!pip install --upgrade tabnet

In [1]:
#!pip install tabnet[gpu]
!pip install tabnet[cpu]

In [2]:
#ocultando mensagens de warning
import warnings
warnings.filterwarnings("ignore")

<hr>

## Download dos conjuntos de dados em formato .csv

In [3]:
import pandas as pd

In [4]:
#!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset-3/master/training_dataset.csv
df_training_dataset = pd.read_csv('./dataset/training_dataset.csv')
df_training_dataset.tail()

,id,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,importante_ter_certificado,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,categoria
15852,620397030.0,Bacharelado,UFF,Outros,Borracha,0.0,0.0,1.0,8.0,NaN,10.0,NaN,NaN,NaN,perfil6
15853,229931283.0,Bacharelado,NaN,Advogado,NaN,0.0,0.0,1.0,7.0,Linkedin,42.0,17.0,15.0,NaN,perfil5
15854,480674907.0,Tecnólogo,UNIP,Sócio/Dono/Proprietário,NaN,0.0,NaN,1.0,7.0,Outros,30.0,9.0,8.0,0.0,perfil5
15855,559626807.0,SEM FORMAÇÃO,UNIVERSIDADE NOVE DE JULHO,Advogado,Estado,0.0,0.0,1.0,10.0,NaN,226.0,102.0,93.0,1.0,perfil1
15856,743652801.0,NaN,FGV-RJ,NaN,Siderurgica,1.0,1.0,1.0,9.0,Outros,125.0,98.0,97.0,1.0,perfil1


In [5]:
df_training_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15857 entries, 0 to 15856
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          13945 non-null  float64
 1   graduacao                   13950 non-null  object 
 2   universidade                13920 non-null  object 
 3   profissao                   13977 non-null  object 
 4   organizacao                 13961 non-null  object 
 5   pretende_fazer_cursos_lit   13989 non-null  float64
 6   interesse_mba_lit           14003 non-null  float64
 7   importante_ter_certificado  13918 non-null  float64
 8   horas_semanais_estudo       13959 non-null  float64
 9   como_conheceu_lit           13915 non-null  object 
 10  total_modulos               13987 non-null  float64
 11  modulos_iniciados           14044 non-null  float64
 12  modulos_finalizados         13924 non-null  float64
 13  certificados                139

In [6]:
df_training_dataset.nunique()

id                            13945
graduacao                         6
universidade                     21
profissao                        12
organizacao                      11
pretende_fazer_cursos_lit         2
interesse_mba_lit                 2
importante_ter_certificado        1
horas_semanais_estudo             9
como_conheceu_lit                 9
total_modulos                   578
modulos_iniciados               372
modulos_finalizados             339
certificados                     23
categoria                         6
dtype: int64

<hr>

## Detalhamento do desafio: classificação multiclasse

Este é um desafio cujo objetivo de negócio é a segmentação dos usuários de uma plataforma de ensino. Para tal, podemos utilizar duas abordagens: aprendizado de máquina supervisionado (classificação) ou não-supervisionado (clustering). Neste desafio será aplicada a classificação, pois é disponível um dataset já com "labels", ou em outras palavras, já com exemplos de dados juntamente com a variável alvo.

Na biblioteca scikit-learn temos diversos algoritmos para classificação. O participante é livre para utilizar o framework que desejar para completar esse desafio.

Neste notebook será mostrado um exeplo de uso do algoritmo "Decision Tree" para classificar parte dos estudantes em seis diferentes perfís.

# Atenção!

A coluna-alvo neste desafio é a coluna ``categoria``

<hr>

## Pre-processando o dataset antes do treinamento

### Removendo todas as linhas que possuem algum valor nulos em determinadas colunas

Usando o método Pandas **DataFrame.dropna()** você pode remover todas as linhas nulas do dataset.

Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html

In [7]:
# Exibindo os dados ausentes do conjunto de dados antes da primeira transformação (df)
print("Valores nulos no df_training_dataset antes da transformação DropNA: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

Valores nulos no df_training_dataset antes da transformação DropNA: 

id                            1912
graduacao                     1907
universidade                  1937
profissao                     1880
organizacao                   1896
pretende_fazer_cursos_lit     1868
interesse_mba_lit             1854
importante_ter_certificado    1939
horas_semanais_estudo         1898
como_conheceu_lit             1942
total_modulos                 1870
modulos_iniciados             1813
modulos_finalizados           1933
certificados                  1878
categoria                        0
dtype: int64



In [8]:
# Aplicando a função para deletar todas as linhas com valor NaN na coluna ``certificados'' e ``total_modulos'':
df_training_dataset = df_training_dataset.dropna(axis='index', how='any', subset=['id', 'importante_ter_certificado'])

In [9]:
# Exibindo os dados ausentes do conjunto de dados após a primeira transformação (df)
print("Valores nulos no df_training_dataset após a transformação DropNA: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

Valores nulos no df_training_dataset após a transformação DropNA: 

id                               0
graduacao                     1467
universidade                  1507
profissao                     1441
organizacao                   1417
pretende_fazer_cursos_lit     1416
interesse_mba_lit             1418
importante_ter_certificado       0
horas_semanais_estudo         1474
como_conheceu_lit             1498
total_modulos                 1460
modulos_iniciados             1411
modulos_finalizados           1460
certificados                  1434
categoria                        0
dtype: int64



### Processando valores NaN com o SimpleImputer do sklearn

Para os valores NaN, usaremos a substituição pela constante 0 como **exemplo**.

Você pode escolher a estratégia que achar melhor para tratar os valores nulos :)

Docs: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html?highlight=simpleimputer#sklearn.impute.SimpleImputer

In [10]:
from sklearn.impute import SimpleImputer
import numpy as np


impute_zeros = SimpleImputer(
    missing_values=np.nan,
    strategy='constant',
    fill_value=0,
    verbose=0,
    copy=True
)

In [11]:
# Exibindo os dados ausentes do conjunto de dados antes da primeira transformação (df)
print("Valores nulos no df_training_dataset antes da transformação SimpleImputer: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

# Aplicando a transformação ``SimpleImputer`` no conjunto de dados base
impute_zeros.fit(X=df_training_dataset)

# Reconstruindo um Pandas DataFrame com os resultados
df_training_dataset_imputed = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_training_dataset
    ),
    columns=df_training_dataset.columns
)

# Exibindo os dados ausentes do conjunto de dados após a primeira transformação (df)
print("Valores nulos no df_training_dataset após a transformação SimpleImputer: \n\n{}\n".format(df_training_dataset_imputed.isnull().sum(axis = 0)))

Valores nulos no df_training_dataset antes da transformação SimpleImputer: 

id                               0
graduacao                     1467
universidade                  1507
profissao                     1441
organizacao                   1417
pretende_fazer_cursos_lit     1416
interesse_mba_lit             1418
importante_ter_certificado       0
horas_semanais_estudo         1474
como_conheceu_lit             1498
total_modulos                 1460
modulos_iniciados             1411
modulos_finalizados           1460
certificados                  1434
categoria                        0
dtype: int64

Valores nulos no df_training_dataset após a transformação SimpleImputer: 

id                            0
graduacao                     0
universidade                  0
profissao                     0
organizacao                   0
pretende_fazer_cursos_lit     0
interesse_mba_lit             0
importante_ter_certificado    0
horas_semanais_estudo         0
como_conheceu_lit    

### Eliminando colunas indesejadas

Vamos **demonstrar** abaixo como usar o método **DataFrame.drop()**.

Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [12]:
df_training_dataset_imputed.tail()

,id,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,importante_ter_certificado,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,categoria
12235,620397030.0,Bacharelado,UFF,Outros,Borracha,0.0,0.0,1.0,8.0,0,10.0,0.0,0.0,0.0,perfil6
12236,229931283.0,Bacharelado,0,Advogado,0,0.0,0.0,1.0,7.0,Linkedin,42.0,17.0,15.0,0.0,perfil5
12237,480674907.0,Tecnólogo,UNIP,Sócio/Dono/Proprietário,0,0.0,0.0,1.0,7.0,Outros,30.0,9.0,8.0,0.0,perfil5
12238,559626807.0,SEM FORMAÇÃO,UNIVERSIDADE NOVE DE JULHO,Advogado,Estado,0.0,0.0,1.0,10.0,0,226.0,102.0,93.0,1.0,perfil1
12239,743652801.0,0,FGV-RJ,0,Siderurgica,1.0,1.0,1.0,9.0,Outros,125.0,98.0,97.0,1.0,perfil1


In [13]:
df_training_dataset_rmcolumns = df_training_dataset_imputed.drop(columns=['id', 'importante_ter_certificado', 
                                                                          'organizacao', 'universidade'], inplace=False)

In [14]:
df_training_dataset_rmcolumns.head()

,graduacao,profissao,pretende_fazer_cursos_lit,interesse_mba_lit,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,categoria
0,Especialização,Analista Senior,1.0,1.0,6.0,Google,28.0,5.0,4.0,0.0,perfil6
1,MBA,Supervisor,0.0,0.0,6.0,0,66.0,36.0,34.0,0.0,perfil2
2,Bacharelado,Outros,1.0,1.0,10.0,Saint Paul,27.0,27.0,27.0,0.0,perfil2
3,SEM FORMAÇÃO,Outros,0.0,1.0,5.0,Instagram,29.0,10.0,6.0,0.0,perfil5
4,Tecnólogo,SEM EXPERIÊNCIA,0.0,0.0,7.0,Google,67.0,49.0,44.0,0.0,perfil2


# Atenção!

As colunas removidas acima são apenas para fim de exemplo, você pode usar as colunas que quiser e inclusive criar novas colunas com dados que achar importantes!


### Tratamento de de variáveis categóricas

Como mencionado antes, os computadores não são bons com variáveis "categóricas" (ou strings).

Dado uma coluna com variável categórica, o que podemos realizar é a codificação dessa coluna em múltiplas colunas contendo variáveis binárias. Esse processo é chamado de "one-hot-encoding" ou "dummy encoding". Se você não é familiarizado com esses termos, você pode pesquisar mais sobre isso na internet :)

In [15]:
# Tratando variáveis categóricas com o método Pandas ``get_dummies()''
df_training = pd.get_dummies(df_training_dataset_rmcolumns, columns=['profissao', 'graduacao', 'como_conheceu_lit'])
df_training.tail()

,pretende_fazer_cursos_lit,interesse_mba_lit,horas_semanais_estudo,total_modulos,modulos_iniciados,modulos_finalizados,certificados,categoria,profissao_0,profissao_Advogado,...,como_conheceu_lit_0,como_conheceu_lit_Facebook,como_conheceu_lit_Google,como_conheceu_lit_Instagram,como_conheceu_lit_Linkedin,como_conheceu_lit_Minha empresa - benefício LITpass,como_conheceu_lit_Mídia (revista/jornal/web),como_conheceu_lit_Outros,como_conheceu_lit_Saint Paul,como_conheceu_lit_YouTube
12235,0.0,0.0,8.0,10.0,0.0,0.0,0.0,perfil6,0,0,...,1,0,0,0,0,0,0,0,0,0
12236,0.0,0.0,7.0,42.0,17.0,15.0,0.0,perfil5,0,1,...,0,0,0,0,1,0,0,0,0,0
12237,0.0,0.0,7.0,30.0,9.0,8.0,0.0,perfil5,0,0,...,0,0,0,0,0,0,0,1,0,0
12238,0.0,0.0,10.0,226.0,102.0,93.0,1.0,perfil1,0,1,...,1,0,0,0,0,0,0,0,0,0
12239,1.0,1.0,9.0,125.0,98.0,97.0,1.0,perfil1,1,0,...,0,0,0,0,0,0,0,1,0,0


# Atenção!

A coluna **categoria** deve ser mantida como uma string. Você não precisa processar/codificar a variável-alvo.

<hr>

## Treinando um classificador com base em uma árvore de decisão

### Selecionando FEATURES e definindo a variável TARGET

In [16]:
df_training.columns

Index(['pretende_fazer_cursos_lit', 'interesse_mba_lit',
       'horas_semanais_estudo', 'total_modulos', 'modulos_iniciados',
       'modulos_finalizados', 'certificados', 'categoria', 'profissao_0',
       'profissao_Advogado', 'profissao_Analista', 'profissao_Analista Senior',
       'profissao_Assessor', 'profissao_Coordenador', 'profissao_Diretor',
       'profissao_Engenheiro', 'profissao_Gerente', 'profissao_Outros',
       'profissao_SEM EXPERIÊNCIA', 'profissao_Supervisor',
       'profissao_Sócio/Dono/Proprietário', 'graduacao_0',
       'graduacao_Bacharelado', 'graduacao_Especialização',
       'graduacao_Licenciatura', 'graduacao_MBA', 'graduacao_SEM FORMAÇÃO',
       'graduacao_Tecnólogo', 'como_conheceu_lit_0',
       'como_conheceu_lit_Facebook', 'como_conheceu_lit_Google',
       'como_conheceu_lit_Instagram', 'como_conheceu_lit_Linkedin',
       'como_conheceu_lit_Minha empresa - benefício LITpass',
       'como_conheceu_lit_Mídia (revista/jornal/web)',
       'como_c

In [17]:
features = df_training[
    [
      
     'pretende_fazer_cursos_lit', 'interesse_mba_lit',
       'horas_semanais_estudo', 'total_modulos', 'modulos_iniciados',
       'modulos_finalizados', 'certificados', 'profissao_0',
       'profissao_Advogado', 'profissao_Analista', 'profissao_Analista Senior',
       'profissao_Assessor', 'profissao_Coordenador', 'profissao_Diretor',
       'profissao_Engenheiro', 'profissao_Gerente', 'profissao_Outros',
       'profissao_SEM EXPERIÊNCIA', 'profissao_Supervisor',
       'profissao_Sócio/Dono/Proprietário', 'graduacao_0',
       'graduacao_Bacharelado', 'graduacao_Especialização',
       'graduacao_Licenciatura', 'graduacao_MBA', 'graduacao_SEM FORMAÇÃO',
       'graduacao_Tecnólogo', 'como_conheceu_lit_0',
       'como_conheceu_lit_Facebook', 'como_conheceu_lit_Google',
       'como_conheceu_lit_Instagram', 'como_conheceu_lit_Linkedin',
       'como_conheceu_lit_Minha empresa - benefício LITpass',
       'como_conheceu_lit_Mídia (revista/jornal/web)',
       'como_conheceu_lit_Outros', 'como_conheceu_lit_Saint Paul',
       'como_conheceu_lit_YouTube'
    ]
]
target = df_training['categoria']  ## NÃO TROQUE O NOME DA VARIÁVEL TARGET.

### Dividindo nosso conjunto de dados em conjuntos de treinamento e teste

In [53]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=133)

### Escolhendo o melhor modelo

In [49]:
# Import dos módulos
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from tabnet import TabNet, TabNetClassifier
from tabnet import StackedTabNetClassifier


# Carregando features e Target
X = features
Y = target

# Definindo os valores para o número de folds
num_folds = 20
seed = 7


# Preparando a lista de modelos
modelos = []
#modelos.append(('LR', LogisticRegression()))
#modelos.append(('LDA', LinearDiscriminantAnalysis()))
#modelos.append(('SVM', SVC()))
#modelos.append(('NB', GaussianNB()))
#modelos.append(('KNN', KNeighborsClassifier()))

modelos.append(('RanF', RandomForestClassifier(bootstrap=False, max_depth=30, max_features='auto',
                        min_samples_split=10, min_weight_fraction_leaf=0,
                        n_estimators=100, warm_start=True)))

#modelos.append(('SGD', SGDClassifier()))
#modelos.append(('ExtraTree', ExtraTreesClassifier()))
modelos.append(('GBoost', GradientBoostingClassifier()))
#modelos.append(('AdaBoost', AdaBoostClassifier()))
#modelos.append(('DesTree', DecisionTreeClassifier()))
#modelos.append(('MLP', MLPClassifier(hidden_layer_sizes=(100,200,100,200,100 ), activation='logistic', 
                                     #solver='adam', alpha=0.0001)))

modelos.append(('TabN', TabNetClassifier(feature_columns = None, num_classes = 6,
                                        feature_dim = 64, output_dim = 32, num_features = len(features),
                                        num_decision_steps = 5, relaxation_factor = 1.5,
                                        sparsity_coefficient = 1e-5, batch_momentum = 0.98,
                                        virtual_batch_size = None, norm_type = 'group',
                                        num_groups = -1)))

# Avaliando cada modelo em um loop
resultados = []
nomes = []

for nome, modelo in modelos:
    kfold = KFold(n_splits = num_folds, random_state = seed)
    cv_results = cross_val_score(modelo, X, Y, cv = kfold, scoring = 'accuracy')
    resultados.append(cv_results)
    nomes.append(nome)
    msg = "%s: %f (%f)" % (nome, cv_results.mean(), cv_results.std())
    print(msg)


# Boxplot para comparar os algoritmos
fig = plt.figure()
fig.suptitle('Comparação de Algoritmos de Classificação')
ax = fig.add_subplot(111)
plt.boxplot(resultados)
ax.set_xticklabels(nomes)
plt.show()

[TabNet]: 32 features will be used for decision steps.
RanF: 0.808088 (0.019969)
GBoost: 0.809886 (0.020696)


TypeError: Cannot clone object '<tabnet.tabnet.TabNetClassifier object at 0x0000014A122E95C8>' (type <class 'tabnet.tabnet.TabNetClassifier'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

### Treinando uma árvore de decisão

In [58]:
# Método para creacion de modelos basados en arbol de desición
from sklearn.ensemble import GradientBoostingClassifier


#dtc = GradientBoostingClassifier(max_depth=3, max_features='auto',
                       #min_samples_split=10, min_weight_fraction_leaf=0.001,
                       #n_estimators=200, warm_start=True).fit(X_train, y_train)



dtc = TabNetClassifier(feature_columns = None, num_classes = 6,
                                        feature_dim = 64, output_dim = 32, num_features = len(features),
                                        num_decision_steps = 5, relaxation_factor = 1.5,
                                        sparsity_coefficient = 1e-5, batch_momentum = 0.98,
                                        virtual_batch_size = None, norm_type = 'group',
                                        num_groups = -1, dynamic=True)
new_model . load_weights ( 'pesos' ) X_train , _ = next ( iter ( tf_dataset ))   # Supondo que gere uma tupla (x, y). modelo . prever ( x )


SyntaxError: invalid syntax (<ipython-input-58-37faccdee665>, line 17)

In [57]:
TabNet.feature_selection_masks

### Fazendo previsões na amostra de teste

In [ ]:
y_pred = dtc.predict(X_test)
print(y_pred)

### Analisando a qualidade do modelo através da matriz de confusão

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools


def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=True):
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if cmap is None:
        cmap = plt.get_cmap('Blues')
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix


plot_confusion_matrix(confusion_matrix(y_test, y_pred), ['parfil1', 'perfil2', 'perfil3', 'perfil4', 'perfil5', 'perfil6'])

<hr>

## Scoring dos dados necessários para entregar a solução

Como entrega da sua solução, esperamos os resultados classificados no seguinte dataset chamado "to_be_scored.csv":

### Download da "folha de respostas"

In [ ]:
#!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset-3/master/to_be_scored.csv
df_to_be_scored = pd.read_csv('to_be_scored.csv')
df_to_be_scored.tail()

# Atenção!

O dataframe ``to_be_scored`` é a sua "folha de respostas". Note que a coluna "categoria" não existe nessa amostra, que não pode ser então utilizada para treino de modelos de aprendizado supervisionado.

In [ ]:
df_to_be_scored.info()

<hr>

# Atenção!

# Para poder aplicar seu modelo e classificar a folha de respostas, você precisa primeiro aplicar as mesmas transformações com colunas que você aplicou no dataset de treino.

# Não remova ou adicione linhas na folha de respostas. 

# Não altere a ordem das linhas na folha de respostas.

# Ao final, as 1000 entradas devem estar classificadas, com os valores previstos em uma coluna chamada "target"

<hr>

Na célula abaixo, repetimos rapidamente os mesmos passos de pré-processamento usados no exemplo dado com árvore de decisão

In [ ]:
# 1 - Removendo linhas com valores NaN em "certificados" e "total_modulos"
df_to_be_scored_1 = df_to_be_scored.dropna(axis='index', how='any', subset=['id'])

# 2 - Inputando zeros nos valores faltantes
impute_zeros.fit(X=df_to_be_scored_1)
df_to_be_scored_2 = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_to_be_scored_1
    ),
    columns=df_to_be_scored_1.columns
)

# 3 - Remoção de colunas
df_to_be_scored_3 = df_to_be_scored_2.drop(columns=['id', 'importante_ter_certificado','organizacao', 'universidade'], inplace=False)

# 4 - Encoding com "dummy variables"
df_to_be_scored_4 = pd.get_dummies(df_to_be_scored_3, columns=['profissao', 'graduacao', 'como_conheceu_lit'])

df_to_be_scored_4.tail()

<hr>

Pode ser verificado abaixo que as colunas da folha de resposta agora são idênticas às que foram usadas para treinar o modelo:

In [ ]:
df_training[
    [
       'pretende_fazer_cursos_lit', 'interesse_mba_lit',
       'horas_semanais_estudo', 'total_modulos', 'modulos_iniciados',
       'modulos_finalizados', 'certificados', 'profissao_0',
       'profissao_Advogado', 'profissao_Analista', 'profissao_Analista Senior',
       'profissao_Assessor', 'profissao_Coordenador', 'profissao_Diretor',
       'profissao_Engenheiro', 'profissao_Gerente', 'profissao_Outros',
       'profissao_SEM EXPERIÊNCIA', 'profissao_Supervisor',
       'profissao_Sócio/Dono/Proprietário', 'graduacao_0',
       'graduacao_Bacharelado', 'graduacao_Especialização',
       'graduacao_Licenciatura', 'graduacao_MBA', 'graduacao_SEM FORMAÇÃO',
       'graduacao_Tecnólogo', 'como_conheceu_lit_0',
       'como_conheceu_lit_Facebook', 'como_conheceu_lit_Google',
       'como_conheceu_lit_Instagram', 'como_conheceu_lit_Linkedin',
       'como_conheceu_lit_Minha empresa - benefício LITpass',
       'como_conheceu_lit_Mídia (revista/jornal/web)',
       'como_conheceu_lit_Outros', 'como_conheceu_lit_Saint Paul',
       'como_conheceu_lit_YouTube'
        
    ]
].columns

In [ ]:
df_to_be_scored_4.columns

# Atenção

Para todas colunas que não existirem no "df_to_be_scored", você pode usar a técnica abaixo para adicioná-las:

In [ ]:
df_to_be_scored_4['como_conheceu_lit_0'] = 0

In [ ]:
y_pred = dtc.predict(df_to_be_scored_4)
df_to_be_scored_4['target'] = y_pred
df_to_be_scored_4.tail()

### Salvando a folha de respostas como um arquivo .csv para ser submetido

In [ ]:
#project.save_data(file_name="results.csv", data=df_to_be_scored_4.to_csv(index=False))

data=df_to_be_scored_4.to_csv("results.csv")

# Atenção

# A execução da célula acima irá criar um novo "data asset" no seu projeto no Watson Studio. Você precisará realizar o download deste arquivo juntamente com este notebook e criar um arquivo zip com os arquivos **results.csv** e **notebook.ipynb** para submissão. (os arquivos devem estar nomeados desta forma)

<hr>

## Parabéns!

Se você já está satisfeito com a sua solução, vá até a página abaixo e envie os arquivos necessários para submissão.

# https://lit.maratona.dev
